In [ ]:
import pandas as pd
import json
import time
import random
import copy

with open("test_data.json", "r") as f:
    test_data = json.loads(f.read())
df = pd.DataFrame(test_data)

def random_date():
    start = time.mktime(time.strptime("2015-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    end = time.mktime(time.strptime("2026-01-01 00:00:00", "%Y-%m-%d %H:%M:%S"))
    random_date = start + random.random() * (end - start)
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(random_date))

random_dates = [random_date() for x in range(len(df["index"]))]
df["timestamp"] = random_dates

df.dropna()
df2 = copy.copy(df)
df3 = pd.concat([df, df2])

with open("data2.csv", "w") as f:
    df3.to_csv(f, index=False)

In [2]:
import tomllib
with open("settings.toml", "rb") as f:
    config = tomllib.load(f)

with open("sensors.toml", "rb") as f:
    sensors = tomllib.load(f)

In [24]:
# Generate dummy data
import datetime 
import random

index_name = "#"
timestamp_name = "Date-Time (CEST)"
temperature_name = "Temperatur, °C"

def generate_data(amt_days=30, daily_samples=144):
    data = {
        index_name: [x for x in range(amt_days*144)],
        timestamp_name: [],
        temperature_name: []
    }
    
    cur_date = datetime.datetime(2015, 3, 1, 0, 0, 0)
    step_minutes = (24*60)/daily_samples
    time_to_add = datetime.timedelta(minutes=step_minutes)

    for i in data[index_name]:
        data[timestamp_name].append(cur_date.strftime(config["formats"]["time_format"]))
        data[temperature_name].append(random.random())
        cur_date += time_to_add

    print(f"{index_name}: {len(data[index_name])}, {timestamp_name}: {len(data[timestamp_name])}, {temperature_name}: {len(data[temperature_name])}")
    return data


In [25]:
data = generate_data()
df = pd.DataFrame(data)

with open("data3.csv", "w", encoding="utf-8") as f:
    df.to_csv(f, index=False)

#: 4320, Date-Time (CEST): 4320, Temperatur, °C: 4320


In [25]:
# Read excel files
import pandas as pd
import glob
import re
import time

path_to_files = r"C:\Users\kevin\Documents\Code\angelverein\Daten FGV"
data_paths = glob.glob(path_to_files+"\\FGV_*.xlsx", recursive=True)
print("Found", len(data_paths), "files")

sensors_chunks = {key: [] for key in sensors.keys()}

stime = time.perf_counter()

# To be tested
for idx, file in enumerate(data_paths):
    sensor_name = re.search(r"FGV_\d+", file).group()
    if not sensor_name in sensors.keys():
        raise(NameError(f"Trying to read sensor {sensor_name} which is not defined in sensors.toml"))
    print("Reading sensor", sensor_name, f"({idx+1}/{len(data_paths)})")
    df = pd.read_excel(file)

    idxcol, timecol, tmpcol = None, None, None
    for idx, col in enumerate(df.columns):
        print(idx, col)
        if config["names"]["index_column"] in col:
            idxcol = col
            # print("Found index column:", col)
        elif config["names"]["timestamp_column"] in col:
            timecol = col
            # print("Found timestamp column:", col)
        elif config["names"]["temperature_column"] in col:
            tmpcol = col
            # print("Found temperature column:", col)
        else: raise(IndexError(f"Found unknown column: {col}"))

    df[timecol] = pd.to_datetime(df[timecol], format=config["formats"]["time_format"])
    df.sort_values(timecol, ascending=False, inplace=True)
    sensors_chunks[sensor_name].append({"df": df, "idxcol": idxcol, "timecol": timecol, "tmpcol": tmpcol})

# Use topmost entry (newest timestamp)
searchfunc = lambda x: x["df"].iloc[0][x["timecol"]]
# Sort chunks after newest newest entry
for key in sensors_chunks.keys():
    sensors_chunks[key].sort(key=searchfunc, reverse=True)

# sensors_chunks should now contain all the sensor files, sorted for all sensors read with newest data at the top

ttime = time.perf_counter() - stime
print(f"Took {ttime:.2f}s")

Found 16 files
Reading sensor FGV_01 (1/16)
0 #
1 Date-Time (CET/CEST)
2 Temperature , °C
Reading sensor FGV_02 (2/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_02 (3/16)
0 #
1 Date-Time (CET/CEST)
2 Temperature , °C
Reading sensor FGV_03 (4/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_03 (5/16)
0 #
1 Date-Time (CET/CEST)
2 Temperature , °C
Reading sensor FGV_04 (6/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_05 (7/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_06 (8/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_07 (9/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_08 (10/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_08 (11/16)
0 #
1 Date-Time (CET/CEST)
2 Temperature (°C) 
Reading sensor FGV_09 (12/16)
0 #
1 Date-Time (CEST)
2 Temperature (°C) 
Reading sensor FGV_09 (13/16)
0 #
1 Date-Time (CET/CEST)
2 Temperature (°C) 
Reading sensor FGV_10 (14/16

In [81]:
dt = pd.to_datetime(sensors_chunks["FGV_01"][0]["df"][:1][sensors_chunks["FGV_01"][0]["timecol"]])
dt

43877   2025-02-12 15:50:00
Name: Date-Time (CET/CEST), dtype: datetime64[ns]

In [23]:
for key in sensors_chunks.keys():
    for chunks in sensors_chunks[key]:
        for col in chunks["df"].columns:
            print(col, end=" ")
        print(f": {chunks["idxcol"]}, {chunks["timecol"]}, {chunks["tmpcol"]}")

# Date-Time (CET/CEST) Temperature , °C : #, Date-Time (CET/CEST), Temperature , °C
# Date-Time (CET/CEST) Temperature , °C : #, Date-Time (CET/CEST), Temperature , °C
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CET/CEST) Temperature , °C : #, Date-Time (CET/CEST), Temperature , °C
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CET/CEST) Temperature (°C)  : #, Date-Time (CET/CEST), Temperature (°C) 
# Date-Time (CEST) Temperature (°C)  : #, Date-Time (CEST), Temperature (°C) 
# Date-Time (CET/CEST) Temperature (°C)  : #, Date-Time (CET/CEST), Temperature (°C) 
# Date-Time (CEST) Temperature

In [ ]:
stime = time.perf_counter()

combined_sensors = []
for key in sensors.keys():
    chunks = sensors_chunks[key]
    chunk_dfs = []
    for chunk in chunks:
        chunk["df"].drop(chunk["idxcol"], axis=1, inplace=True)
        chunk["df"]["Datum"] = chunk["df"][chunk["timecol"]].dt.date
        chunk["df"]["Jahr"] = chunk["df"][chunk["timecol"]].dt.year
        chunk["df"]["Monat"] = chunk["df"][chunk["timecol"]].dt.month
        chunk["df"]["Tag"] = chunk["df"][chunk["timecol"]].dt.day
        chunk["df"]["Uhrzeit"] = chunk["df"][chunk["timecol"]].dt.time
        chunk["df"]["Sensor"] = key
        chunk["df"]["Standort"] = sensors[key]["location"]
        chunk["df"].drop(chunk["timecol"], axis=1, inplace=True)
        chunk["df"].rename(columns={chunk["tmpcol"]: "Temperatur"}, inplace=True)
        chunk_dfs.append(chunk["df"])
        for col in chunk["df"].columns:
            print(col, end=" ")
        print(" ")
    combined_sensors.append(pd.concat(chunk_dfs))

combined_sensors = pd.concat(combined_sensors)

with open("all_data.csv", "w") as f:
    combined_sensors.to_csv(f, index=False)

ttime = time.perf_counter() - stime
print(f"Finished in {ttime:.2f}s")

Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Temperatur Datum Jahr Monat Tag Uhrzeit Sensor Standort  
Finished in 2.20s


In [84]:
sensors_chunks["FGV_01"][0]["df"]["Time"] = sensors_chunks["FGV_01"][0]["df"][sensors_chunks["FGV_01"][0]["timecol"]].dt.time
sensors_chunks["FGV_01"][0]["df"]

,#,Date-Time (CET/CEST),"Temperature , °C",Day,Time
43877,44046,2025-02-12 15:50:00,7.161592,12,15:50:00
43876,44045,2025-02-12 15:40:00,6.560986,12,15:40:00
43875,44044,2025-02-12 15:30:00,6.518086,12,15:30:00
43874,44043,2025-02-12 15:20:00,6.518086,12,15:20:00
43873,44042,2025-02-12 15:10:00,6.518086,12,15:10:00
...,...,...,...,...,...
4,173,2024-04-14 00:40:00,12.824443,14,00:40:00
3,172,2024-04-14 00:30:00,12.910244,14,00:30:00
2,171,2024-04-14 00:20:00,12.996045,14,00:20:00
1,170,2024-04-14 00:10:00,12.996045,14,00:10:00
